In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.
C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 

In [2]:
data = pd.read_csv("fake_real_cnn.csv",  encoding='utf-8')

In [3]:
data.head(10)

,Unnamed: 0,text,Label
0,2899,ا مقتل ثلاثة جنود المان في افغانستان مقتل ثلا...,Real
1,781,حركة العدل والمساواة تفرج عن ستين من عناصر ال...,Real
2,3700,ا باكستان المحكمة العليا تهد بسجن رئيس مكتب م...,Real
3,3002,الأسطورة الحي ة كريم بنزيما يمت ع الجماهير بل...,fake
4,3116,باحث فرنسي مارين لوبان من نسل النبي محمد,fake
5,2595,انخفاض حاد لمنسوب الشباب لدى اليافعين أثبت مرك...,fake
6,967,أم تستخدم مسدسا صاعقا لإيقاظ ابنها لحضور ا...,fake
7,2320,اليمن مقتل ثلاثة جنود في محافظة شبوة على أيدي...,Real
8,3616,ا بريطانيا وفرنسا تقترحان صندوقا لمناخ في قمة...,Real
9,1161,الاتحاد الأوروبي يحث إسرائيل على وقف الاستيطا...,Real


In [4]:
data.shape

(6895, 3)

In [5]:
data.isnull().sum()

Unnamed: 0    0
text          0
Label         0
dtype: int64

In [6]:
# Set index 
data = data.set_index("Unnamed: 0")

# Print first lines of `df` 
data.head()

,text,Label
Unnamed: 0,,
2899,ا مقتل ثلاثة جنود المان في افغانستان مقتل ثلا...,Real
781,حركة العدل والمساواة تفرج عن ستين من عناصر ال...,Real
3700,ا باكستان المحكمة العليا تهد بسجن رئيس مكتب م...,Real
3002,الأسطورة الحي ة كريم بنزيما يمت ع الجماهير بل...,fake
3116,باحث فرنسي مارين لوبان من نسل النبي محمد,fake


In [7]:
import re
def cleaning (txt):
    
   
    a = ['خاص للحدود',
         ' ليتها ',
         ' سيكون قسم التعليقات متاحاً خلال بضعة أيّام ',
         ' سيكون قسم التعليقات متاحا  خلال بضعة ',
         ' وقعت على الأرض ضحكاً ',
         ' ضحكت دون أن أقع على الأرض ',
         ' مؤلم ',
         ' إبداع ',
         ' تجاوزت الحدود  ',
         ' غبي ',
         'المذكور أعلاه ',
         'سيكون قسم التعليقات متاحاً خلال بضعة أيّام ',
         '!',
         "''",
         ',',
         ':',
         '``',
         '،',
         '؟',
         '||',
         '٪',
         'سيكون قسم التعليقات متاحاً خلال بضعة أيّام',
         ' الحدود ',
         'ليتها',
         'كاريكاتير',
         ' آآآآآآآمين ',
         ' آآآه ',
         ' آآآآآخ ',
         'آآآآآآو ',
         '(',
         '=',
         '.',
         '[]',
         ')',
         ';',
         '{}',
         "“",
         "”",
         '-',
         '"',
         '/',
         "'",
         '&#',
         '#',
         '&',
         '%',
         '{ }',
         ' قد ',
         ' أن ',
         ' إن ',
         ' إلا ',
         ' تماماً ',
         ':'
         
         
        
        "كانون الأول","تشرين الثاني", ''
         ,"تشرين الأول "," أيلول","تموز"," حزيران "," أيار"," نيسان ","كانون الثاني", " شباط ",' أغسطس آب ',' بي بي سي '," آذار ",
         "BBC Arabic", "content=", "الرئيسية", "الشرق الأوسط"
         , "شارك برأيك", "اقتصاد وأعمال", "/>", "علوم وتكنولوجيا","منوعات", "العالم", "سؤال وجواب:"
         , " راديو وتلفزيون"
         ,  "GMT",'رياضة', " - - " ,'؟', '.',"''",',',':','``','=','،','-', 
         " آخر تحديث","بالصور","تشرين", "الثاني", "2009", "2010", "حزيران", "BBCArabic.com", "كانون", ' ! '
        ]
    for word in a:
        txt = txt.replace(word,'')
       
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    txt = re.sub(p_tashkeel,"", txt)#remove tashkeel (diactritics)
    txt = re.sub(r'\d+', ' ', txt) #substituting digits with 'number'
    txt = re.sub(r'[a-zA-Z]+', ' ', txt, flags=re.I) #deleting english letters
    txt = re.sub(r'\W', ' ', txt, flags = re.I) # remove sepcial characters
    txt = re.sub(r'\s+', ' ', txt) #Substituting multiple spaces with single space
#     txt = re.sub(r'(?:^| )\w(?:$| )', ' ', txt) #Substituting single characters
#     txt = re.sub(r'(\s+[a-zA-Z]\s+)', ' ', txt) #Substituting single characters
    



    return txt

In [8]:
data['text'] = data['text'].apply(cleaning)

In [22]:
from sklearn.model_selection import train_test_split

# split complete dataset in train-test with ratio 80-20 respectively

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['Label'], test_size=0.2, random_state=1)


# again split train data in train-validation with 90-10 ratio respectively

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator
class Text2VECTORTransformer(BaseEstimator):

    def __init__(self):
        self._model = CountVectorizer(max_features=1500)
        pass

    def fit(self, X_train, y_train=None):
        #X_train = X_train.apply(lambda x : cleaning_hudoodtext2(x))
        self._model.fit(X_train)
        return self

    def transform(self, X_train):
        return self._model.transform(X_train)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator
class Text2tfTransformer(BaseEstimator):

    def __init__(self):
        self._model = TfidfVectorizer(max_features=1500)
        pass

    def fit(self, X_train, y_train=None):
        #X_train = X_train.apply(lambda x : cleaning_hudoodtext2(x))
        self._model.fit(X_train)
        return self

    def transform(self, X_train):
        return self._model.transform(X_train)

In [25]:
# tf

import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

pl_xgb_tf_idf = Pipeline(steps=[('COUNT',Text2tfTransformer()),
                         ('xgboost', xgb.XGBClassifier(objective='binary:logitraw'))])
pl_xgb_tf_idf.fit(X_train,  y_train)
preds = pl_xgb_tf_idf.predict(X_val)

C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
print(classification_report(preds, y_val, target_names=['fake', 'Real'],digits=4))
acc = metrics.accuracy_score(y_val,preds)
acc

             precision    recall  f1-score   support

       fake     0.9743    0.9601    0.9672       276
       Real     0.9607    0.9746    0.9676       276

avg / total     0.9675    0.9674    0.9674       552



0.967391304347826

In [27]:
preds_test = pl_xgb_tf_idf.predict(X_test)
print(classification_report(preds_test, y_test, target_names=['fake', 'Real'],digits=4))
acc_test = metrics.accuracy_score(y_test,preds_test)
acc_test

             precision    recall  f1-score   support

       fake     0.9780    0.9807    0.9794       726
       Real     0.9785    0.9755    0.9770       653

avg / total     0.9782    0.9782    0.9782      1379



C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9782451051486585

In [28]:
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

pl_xgb_tf_idf = Pipeline(steps=[('COUNT',Text2VECTORTransformer()),
                         ('xgboost', xgb.XGBClassifier(objective='binary:logitraw'))])
pl_xgb_tf_idf.fit(X_train,  y_train)
preds = pl_xgb_tf_idf.predict(X_val)

C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [29]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
print(classification_report(preds, y_val, target_names=['fake', 'Real'],digits=4))
acc = metrics.accuracy_score(y_val,preds)
acc

             precision    recall  f1-score   support

       fake     0.9669    0.9359    0.9512       281
       Real     0.9357    0.9668    0.9510       271

avg / total     0.9516    0.9511    0.9511       552



0.9510869565217391

In [30]:
preds_test = pl_xgb_tf_idf.predict(X_test)
print(classification_report(preds_test, y_test, target_names=['fake', 'Real'],digits=4))
acc_test = metrics.accuracy_score(y_test,preds_test)
acc_test

             precision    recall  f1-score   support

       fake     0.9794    0.9727    0.9760       733
       Real     0.9693    0.9768    0.9730       646

avg / total     0.9747    0.9746    0.9746      1379



C:\Users\sadan\AppData\Local\Continuum\anaconda3\envs\old_env\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9746192893401016